In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/train1.csv",encoding="ISO-8859-1")

<ipython-input-4-715d4c072053>:1: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264

In [5]:
df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.describe()

,Unnamed: 524,Unnamed: 529,Unnamed: 534,Unnamed: 539,Unnamed: 543,Unnamed: 556,Unnamed: 561,Unnamed: 564,Unnamed: 565,Unnamed: 566,...,Unnamed: 601,Unnamed: 607,Unnamed: 612,Unnamed: 617,Unnamed: 621,Unnamed: 625,Unnamed: 654,Unnamed: 675,Unnamed: 683,Unnamed: 685
count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1992.0,1992.0,1992.0,1992.0,1992.0,1972.0,1990.0,183.0,187.0,194.0,...,1992.0,1975.0,1975.0,1975.0,1977.0,1979.0,1977.0,1986.0,1991.0,1.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1992.0,1992.0,1992.0,1992.0,1992.0,1972.0,1990.0,183.0,187.0,194.0,...,1992.0,1975.0,1975.0,1975.0,1977.0,1979.0,1977.0,1986.0,1991.0,1.0
25%,1992.0,1992.0,1992.0,1992.0,1992.0,1972.0,1990.0,183.0,187.0,194.0,...,1992.0,1975.0,1975.0,1975.0,1977.0,1979.0,1977.0,1986.0,1991.0,1.0
50%,1992.0,1992.0,1992.0,1992.0,1992.0,1972.0,1990.0,183.0,187.0,194.0,...,1992.0,1975.0,1975.0,1975.0,1977.0,1979.0,1977.0,1986.0,1991.0,1.0
75%,1992.0,1992.0,1992.0,1992.0,1992.0,1972.0,1990.0,183.0,187.0,194.0,...,1992.0,1975.0,1975.0,1975.0,1977.0,1979.0,1977.0,1986.0,1991.0,1.0
max,1992.0,1992.0,1992.0,1992.0,1992.0,1972.0,1990.0,183.0,187.0,194.0,...,1992.0,1975.0,1975.0,1975.0,1977.0,1979.0,1977.0,1986.0,1991.0,1.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16426 entries, 0 to 16425
Columns: 686 entries, id to Unnamed: 685
dtypes: float64(24), object(662)
memory usage: 86.0+ MB


In [8]:
df.isnull().sum()

,0
id,3607
title,5047
author,6325
text,5688
label,5917
...,...
Unnamed: 681,16425
Unnamed: 682,16425
Unnamed: 683,16425
Unnamed: 684,16425


In [9]:
df=df.fillna('')

In [10]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0
...,...
Unnamed: 681,0
Unnamed: 682,0
Unnamed: 683,0
Unnamed: 684,0


In [11]:
df.columns

Index(['id', 'title', 'author', 'text', 'label', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 676', 'Unnamed: 677', 'Unnamed: 678', 'Unnamed: 679',
       'Unnamed: 680', 'Unnamed: 681', 'Unnamed: 682', 'Unnamed: 683',
       'Unnamed: 684', 'Unnamed: 685'],
      dtype='object', length=686)

In [12]:

df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,,,,,,...,,,,,,,,,,
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,,,,,,...,,,,,,,,,,
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,,,,,,...,,,,,,,,,,
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,,,,,,...,,,,,,,,,,
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,,,,,,...,,,,,,,,,,


In [13]:
columns = ['text', 'label']
df = df[columns]


In [14]:
df.head()

,text,label
0,House Dem Aide: We Didnât Even See Comeyâs...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
port_stem=PorterStemmer()

In [17]:
port_stem

<PorterStemmer>

In [18]:
def stemming(content):
  con=re.sub('[^a-zA-Z]',' ',content)
  con=con.lower()
  con=con.split()
  con=[port_stem.stem(word)for word in con if not word in stopwords.words('english')]
  con=' '.join(con)
  return con

In [19]:
!pip install swifter
import swifter

df['text'] = df['text'].swifter.apply(stemming)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 21.4 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=53d8e54e44f8987f63babb7424d9e25dfd0d546f7c59f0d55e3b52e30904816c
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


Pandas Apply:   0%|          | 0/16426 [00:00<?, ?it/s]

In [20]:
x=df['text'].astype(str)

In [21]:
y=df['label'].astype(str)

In [22]:
y.shape

(16426,)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vect=TfidfVectorizer()


In [27]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [28]:
x_test.shape

(3286, 66911)

In [29]:
from sklearn.tree import DecisionTreeClassifier

In [30]:
model=DecisionTreeClassifier()

In [31]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [32]:
prediction=model.predict(x_test)

In [33]:
prediction

array(['', '', '1', ..., '', '0', '0'], dtype=object)

In [34]:
print("Accuracy is: ",model.score(x_test, y_test)*100)

Accuracy is:  84.20572124163115


In [35]:
import pickle

In [36]:
pickle.dump(vect, open('vect.pkl','wb'))

In [37]:
pickle.dump(model, open('model.pkl','wb'))

In [38]:
vector_form=pickle.load(open('vect.pkl','rb'))

In [39]:
load_model=pickle.load(open('model.pkl','rb'))

In [40]:
def fake_news(news):
  news=stemming(news)
  input_data=[news]
  vector_form1=vector_form.transform(input_data)
  prediction=load_model.predict(vector_form1)
  return prediction


In [43]:
val=fake_news("""House Dem Aide: We Didnâ€™t Even See Comeyâ€™s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license)
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this weekâ€“FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clintonâ€™s email server, the ranking Democrats on the relevant committees didnâ€™t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen.
As we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emails it had recently discovered in order to see if they contained classified information. Not long after this letter went out, Oversight Committee Chairman Jason Chaffetz set the political world ablaze with this tweet. FBI Dir just informed me, "The FBI has learned of the existence of emails that appear to be pertinent to the investigation." Case reopened
â€” Jason Chaffetz (@jasoninthehouse) October 28, 2016
Of course, we now know that this was not the case . Comey was actually saying that it was reviewing the emails in light of â€œan unrelated caseâ€â€“which we now know to be Anthony Weinerâ€™s sexting with a teenager. But apparently such little things as facts didnâ€™t matter to Chaffetz. The Utah Republican had already vowed to initiate a raft of investigations if Hillary winsâ€“at least two yearsâ€™ worth, and possibly an entire termâ€™s worth of them. Apparently Chaffetz thought the FBI was already doing his work for himâ€“resulting in a tweet that briefly roiled the nation before cooler heads realized it was a dud.
But according to a senior House Democratic aide, misreading that letter may have been the least of Chaffetzâ€™ sins. That aide told Shareblue that his boss and other Democrats didnâ€™t even know about Comeyâ€™s letter at the timeâ€“and only found out when they checked Twitter. â€œDemocratic Ranking Members on the relevant committees didnâ€™t receive Comeyâ€™s letter until after the Republican Chairmen. In fact, the Democratic Ranking Members didnâ€™ receive it until after the Chairman of the Oversight and Government Reform Committee, Jason Chaffetz, tweeted it out and made it public.â€
So letâ€™s see if weâ€™ve got this right. The FBI director tells Chaffetz and other GOP committee chairmen about a major development in a potentially politically explosive investigation, and neither Chaffetz nor his other colleagues had the courtesy to let their Democratic counterparts know about it. Instead, according to this aide, he made them find out about it on Twitter.
There has already been talk on Daily Kos that Comey himself provided advance notice of this letter to Chaffetz and other Republicans, giving them time to turn on the spin machine. That may make for good theater, but there is nothing so far that even suggests this is the case. After all, there is nothing so far that suggests that Comey was anything other than grossly incompetent and tone-deaf.
What it does suggest, however, is that Chaffetz is acting in a way that makes Dan Burton and Darrell Issa look like models of responsibility and bipartisanship. He didnâ€™t even have the decency to notify ranking member Elijah Cummings about something this explosive. If that doesnâ€™t trample on basic standards of fairness, I donâ€™t know what does.
Granted, itâ€™s not likely that Chaffetz will have to answer for this. He sits in a ridiculously Republican district anchored in Provo and Orem; it has a Cook Partisan Voting Index of R+25, and gave Mitt Romney a punishing 78 percent of the vote in 2012. Moreover, the Republican House leadership has given its full support to Chaffetzâ€™ planned fishing expedition. But that doesnâ€™t mean we canâ€™t turn the hot lights on him. After all, he is a textbook example of what the House has become under Republican control. And he is also the Second Worst Person in the World. About Darrell Lucus
Darrell is a 30-something graduate of the University of North Carolina who considers himself a journalist of the old school. An attempt to turn him into a member of the religious right in college only succeeded in turning him into the religious right's worst nightmare--a charismatic Christian who is an unapologetic liberal. His desire to stand up for those who have been scared into silence only increased when he survived an abusive three-year marriage. You may know him on Daily Kos as Christian Dem in NC . Follow him on Twitter @DarrellLucus or connect with him on Facebook . Click here to buy Darrell a Mello Yello. Connect """)

In [44]:
if val==['0']:
  print("Reliable: News is not fake ")
else:
  print("Unreliable: News is fake")

Unreliable: News is fake
